In [3]:
import os
from PIL import Image
import numpy as np
from pycocotools.coco import COCO

In [4]:
coco = COCO("../input/data/train.json")

loading annotations into memory...
Done (t=3.82s)
creating index...
index created!


In [7]:
data_path = "../input/data"
seg_path = "../input/data/seg_image"
os.makedirs(seg_path, exist_ok=True)
os.makedirs(os.path.join(seg_path, "batch_01_vt"), exist_ok=True)
os.makedirs(os.path.join(seg_path, "batch_02_vt"), exist_ok=True)
os.makedirs(os.path.join(seg_path, "batch_03"), exist_ok=True)

for i in range(len(coco.imgs)):
    image_id = coco.getImgIds(imgIds=i)
    image_infos = coco.loadImgs(image_id)[0]
    file_name = image_infos["file_name"]
    image = Image.open(os.path.join(data_path, file_name))

    ann_ids = coco.getAnnIds(imgIds=image_infos["id"])
    ann_list = coco.loadAnns(ann_ids)

    mask = np.full((image.height, image.width), 255)
    for i, ann in enumerate(ann_list):
        ann_mask = np.int_(coco.annToMask(ann))
        ann_mask *= ann["category_id"] - 255
        mask += ann_mask
        map_image = Image.fromarray(np.uint8(mask), "L")

    image.close()
    map_image.save(os.path.join(seg_path, file_name[:-4] + ".png"), "PNG")